<a href="https://colab.research.google.com/github/Smartsie/Playground/blob/master/Char_RNN_Thierry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
This code is just for fun to make a text generator by myself, inspired by Karpathy but completely made up by myself
T Martin, 12/11/2021
"""

'\nThis code is just for fun to make a text generator by myself, inspired by Karpathy but completely made up by myself\nT Martin, 12/11/2021\n'

In [8]:
import numpy as np
import pandas as pd
import base64
import requests

In [30]:
# Import the big text data
url = "http://f8cho.free.fr/RABELAIS.txt"


In [132]:
shakespeare_url='https://homl.info/shakespeare'
filepath=tf.keras.utils.get_file('shakespeare.txt',shakespeare_url)
with open(filepath) as f:
  shakespeare_text=f.read()

In [133]:
text=shakespeare_text

In [116]:
def get_text(url):
  '''
    The function get_text will load a txt file at the specified url

    Args:
        url: The url where the text is located. 

    Returns:
        req!: a clean text
  '''

  req = requests.get(url)
  req = req.text
  return req

In [117]:
text=get_text(url)

In [118]:
import keras
import tensorflow as tf

In [136]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)
max_id=len(tokenizer.word_index)
dataset_size=tokenizer.document_count
encoded=np.array(tokenizer.texts_to_sequences([text]))-1

In [137]:
train_size=dataset_size*90//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])
# Transform dataset by using windows
n_steps=100
window_length=n_steps+1
dataset=dataset.window(window_length, shift=1,drop_remainder=True)
dataset=dataset.flat_map(lambda window: window.batch(window_length))
# Shuffle the dataset with a
batch_size=32
dataset=dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:,:,-1],windows[:,1:]))
# One hot encoding
dataset=dataset.map(
    lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch)
)
# prefetching
dataset=dataset.prefetch(1)

In [140]:
model=tf.keras.models.Sequential([
    tf.keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id],dropout=0.2,recurrent_dropout=0.2),
    tf.keras.layers.GRU(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))    
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',run_eagerly=True)


In [141]:
history=model.fit(dataset,epochs=20)

Epoch 1/20


ValueError: ignored

In [101]:
dataset

<PrefetchDataset shapes: ((None, None, 63), (None, None, 33317)), types: (tf.float32, tf.int32)>